<a href="https://colab.research.google.com/github/Kethanvr/Reag-VecotrDB/blob/main/Final_Rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q pymongo sentence-transformers google-generativeai python-dotenv numpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 331.1/331.1 kB 19.9 MB/s eta 0:00:00


In [3]:


import os
import hashlib
import numpy as np
from typing import List, Dict, Any
from sentence_transformers import SentenceTransformer
import google.generativeai as genai
from pymongo import MongoClient
from google.colab import userdata

/usr/local/lib/python3.12/dist-packages/google/colab/_import_hooks/_hook_injector.py:55: FutureWarning: 

All support for the `google.generativeai` package has ended. It will no longer be receiving 
updates or bug fixes. Please switch to the `google.genai` package as soon as possible.
See README for more details:

https://github.com/google-gemini/deprecated-generative-ai-python/blob/main/README.md

  loader.exec_module(module)


In [6]:
GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')  # Your Gemini API key
MONGODB_URI = userdata.get('MONGODB_URI')  # Your MongoDB Atlas connection string

# Configure Gemini
genai.configure(api_key=GEMINI_API_KEY)

In [32]:
class RAGSystem:
    def __init__(self, mongodb_uri: str, db_name: str = "rag_db", collection_name: str = "documents"):
        """Initialize RAG system with BGE embeddings and MongoDB"""

        # 1. Load BGE embedding model (1024 dimensions)
        print("Loading BGE-large model (1024D)...")
        self.embedding_model = SentenceTransformer('BAAI/bge-large-en-v1.5')
        self.embedding_dim = 1024

        # 2. Initialize MongoDB connection
        print("Connecting to MongoDB Atlas...")
        self.client = MongoClient(MONGODB_URI)
        self.db = self.client[db_name]
        self.collection = self.db[collection_name]

        # 3. Initialize Gemini model
        print("Initializing Gemini model...")
        self.llm = genai.GenerativeModel('gemini-2.5-flash-lite')

        print("✅ RAG System initialized successfully!")

    def chunk_text(self, text: str, chunk_size: int = 500, overlap: int = 50) -> List[str]:
        """Split text into overlapping chunks"""
        words = text.split()
        chunks = []

        for i in range(0, len(words), chunk_size - overlap):
            chunk = ' '.join(words[i:i + chunk_size])
            if chunk.strip():
                chunks.append(chunk)

        return chunks

    def generate_hash(self, text: str) -> str:
        """Generate SHA-256 hash for deduplication"""
        return hashlib.sha256(text.encode()).hexdigest()

    def ingest_document(self, text: str, metadata: Dict[str, Any] = None):
        """Ingest a document: chunk, embed, deduplicate, and store"""
        if metadata is None:
            metadata = {}

        # Step 1: Chunk the document
        print(f"Chunking document...")
        chunks = self.chunk_text(text)
        print(f"Created {len(chunks)} chunks")

        # Step 2 & 3: Generate embeddings and hashes
        print("Generating embeddings and hashes...")
        for i, chunk in enumerate(chunks):
            # Generate embedding (1024D vector)
            embedding = self.embedding_model.encode(chunk).tolist()

            # Generate dedup hash
            chunk_hash = self.generate_hash(chunk)

            # Check if chunk already exists (deduplication)
            existing = self.collection.find_one({"hash": chunk_hash})
            if existing:
                print(f"Chunk {i+1} already exists (deduplicated)")
                continue

            # Store in MongoDB
            document = {
                "text": chunk,
                "embedding": embedding,
                "hash": chunk_hash,
                "metadata": {**metadata, "chunk_index": i}
            }

            self.collection.insert_one(document)
            print(f"✓ Stored chunk {i+1}/{len(chunks)}")

        print("✅ Document ingestion complete!")

    def search(self, query: str, top_k: int = 3) -> List[Dict[str, Any]]:
        """Perform semantic search using vector similarity"""
        # Generate query embedding
        query_embedding = self.embedding_model.encode(query).tolist()

        # MongoDB Vector Search Pipeline
        pipeline = [
            {
                "$vectorSearch": {
                    "index": "vector_index",
                    "path": "embedding",
                    "queryVector": query_embedding,
                    "numCandidates": top_k * 10,
                    "limit": top_k
                }
            },
            {
                "$project": {
                    "_id": 0,
                    "text": 1,
                    "metadata": 1,
                    "score": {"$meta": "vectorSearchScore"}
                }
            }
        ]

        try:
            results = list(self.collection.aggregate(pipeline))
            return results
        except Exception as e:
            print(f"⚠️ Search error: {e}")
            print("Note: Make sure vector search index 'vector_index' is created in MongoDB Atlas!")
            return []

    def generate_answer(self, query: str, top_k: int = 3) -> Dict[str, Any]:
        """Generate answer using RAG approach"""
        # Step 1: Retrieve relevant context
        print(f"🔍 Searching for relevant context...")
        search_results = self.search(query, top_k=top_k)

        if not search_results:
            return {
                "answer": "I couldn't find relevant information to answer your question. Please make sure the vector search index is set up in MongoDB Atlas.",
                "sources": [],
                "context_used": [],
                "num_sources": 0  # ADD THIS LINE
            }

        # Step 2: Prepare context
        context_chunks = [result['text'] for result in search_results]
        context = "\n\n".join([f"[{i+1}] {chunk}" for i, chunk in enumerate(context_chunks)])

        # Step 3: Create prompt
        prompt = f"""You are a helpful assistant. Answer the question based ONLY on the provided context.
If the context doesn't contain enough information, say so. Do not make up information.

Context:
{context}

Question: {query}

Answer:"""

        # Step 4: Generate answer using Gemini
        print("🤖 Generating answer with Gemini...")
        response = self.llm.generate_content(prompt)
        answer = response.text

        return {
            "answer": answer,
            "sources": search_results,
            "context_used": context_chunks,
            "num_sources": len(search_results)
        }

In [33]:
# STEP 5: Usage Example
# ============================================================================

print("\n" + "="*80)
print("INITIALIZING RAG SYSTEM")
print("="*80)

# Initialize RAG system
rag = RAGSystem(
    mongodb_uri=MONGODB_URI,
    db_name="rag",
    collection_name="rag-collection"
)

# Example document
sample_document = """
Artificial Intelligence (AI) is transforming the world. Machine learning, a subset of AI,
enables computers to learn from data without explicit programming. Deep learning uses neural
networks with multiple layers to process complex patterns. Natural Language Processing (NLP)
helps machines understand human language. Computer vision allows machines to interpret images
and videos. AI applications include healthcare diagnosis, autonomous vehicles, recommendation
systems, and virtual assistants. The field of AI continues to evolve rapidly with new
breakthroughs in areas like reinforcement learning, generative AI, and transfer learning.
"""

print("\n" + "="*80)
print("INGESTING DOCUMENT")
print("="*80)

rag.ingest_document(
    text=sample_document,
    metadata={"source": "AI Overview", "topic": "Artificial Intelligence"}
)

# Ask a question
print("\n" + "="*80)
print("ASKING QUESTION")
print("="*80)

question = "What is machine learning?"
result = rag.generate_answer(question, top_k=3)

print(f"\n📝 Question: {question}")

# Safe access to result dictionary
if result.get('answer'):
    print(f"\n✅ Answer:\n{result['answer']}")
else:
    print(f"\n⚠️ No answer generated")

# Safely check for sources
num_sources = len(result.get('sources', []))
print(f"\n📚 Used {num_sources} source(s)")

# Safely display context
context_used = result.get('context_used', [])
if context_used:
    print("\n🔍 Retrieved Context:")
    for i, context in enumerate(context_used, 1):
        print(f"\n[{i}] {context[:200]}...")
else:
    print("\n⚠️ No context retrieved - Vector search index might not be set up yet!")
    print("\n👉 Follow the setup instructions below to create the vector index in MongoDB Atlas")
# ============================================================================
# MongoDB Atlas Setup Instructions
# ============================================================================

print("\n" + "="*80)
print("⚠️  IMPORTANT: MongoDB Atlas Vector Search Index Setup")
print("="*80)
print("""
To enable vector search, create an index in MongoDB Atlas:

1. Go to MongoDB Atlas Dashboard
2. Navigate to your cluster > Browse Collections
3. Select database: rag_demo, collection: documents
4. Click "Search Indexes" tab
5. Click "Create Search Index"
6. Choose "JSON Editor"
7. Paste this configuration:

{
  "fields": [
    {
      "type": "vector",
      "path": "embedding",
      "numDimensions": 1024,
      "similarity": "cosine"
    }
  ]
}""")


INITIALIZING RAG SYSTEM
Loading BGE-large model (1024D)...
Connecting to MongoDB Atlas...
Initializing Gemini model...
✅ RAG System initialized successfully!

INGESTING DOCUMENT
Chunking document...
Created 1 chunks
Generating embeddings and hashes...
Chunk 1 already exists (deduplicated)
✅ Document ingestion complete!

ASKING QUESTION
🔍 Searching for relevant context...
🤖 Generating answer with Gemini...

📝 Question: What is machine learning?

✅ Answer:
Machine learning is a subset of AI that enables computers to learn from data without explicit programming.

📚 Used 1 source(s)

🔍 Retrieved Context:

[1] Artificial Intelligence (AI) is transforming the world. Machine learning, a subset of AI, enables computers to learn from data without explicit programming. Deep learning uses neural networks with mul...

⚠️  IMPORTANT: MongoDB Atlas Vector Search Index Setup

To enable vector search, create an index in MongoDB Atlas:

1. Go to MongoDB Atlas Dashboard
2. Navigate to your cluster > Br